In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import dotenv

In [3]:
dotenv.load_dotenv()

True

In [4]:
from typing import TypedDict, Annotated, List, Dict, Literal
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Image
import operator

In [5]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [6]:
from langgraph.graph import START, END, StateGraph
from langgraph.types import Command
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_experimental.utilities import PythonREPL
from langgraph.graph.message import add_messages

In [7]:
#from src.utils_email import send_report_via_email
from src.utils_trans import search_flight_details

In [8]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )

In [7]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [9]:
class State(TypedDict):
    messages: Annotated[List, add_messages]  # Chat history
    outbond_flight: str # ser input
    return_flight: str  # user input
    outbond_date: str  # user input
    return_date: str  # user input

    trip_day: int #from trip info agent
    location: str  #from trip info agent
    arrival_time: str #from trip info agent
    return_time: str #from trip info agent

    splits: List[Dict]
    num_splits: int
    chunk_results: Annotated[List[Dict], operator.add]

    combined_schedule: Dict


### Transportation Research Agent

In [ ]:
@tool
def get_flight_data(
) -> str:
    """

    Retrieve location and time information for a round-trip flight to assist with travel planning.

    Returns:
        start_flight (str): Flight number for the departure flight.
        return_flight (str): Flight number for the return flight.
    """

    result = search_flight_details()
    return json.dumps(result)

In [ ]:
#get_flight_data()

'{"start_flight": {"departure_airport": "Hong Kong International Airport", "arrival_airport": "Osaka Kansai International Airport", "departure_region": "Hong Kong", "arrival_region": "Osaka", "departure_time": "09:11", "estimated_arrival_time": "12:36"}, "return_flight": {"departure_airport": "Osaka Kansai International Airport", "arrival_airport": "Hong Kong International Airport", "departure_region": "Osaka", "arrival_region": "Hong Kong", "departure_time": "09:11", "estimated_arrival_time": "12:41"}}'

In [40]:
flight_data_agent = create_react_agent(
    llm,
    tools=[get_flight_data],
    prompt=get_system_prompt(
        "You are a travel planning assistant, specializing in retrieving and presenting round-trip flight information. Your primary tool, get_flight_data, accepts two flight numbers (departure and return) and returns a JSON string with details including departure and arrival airports, regions, and times."
    ),
)

### Activities (Weather focused) Agent

In [ ]:
@tool
def search_seasonal_activities(city: str, season: str) -> str:
    
    return json.dumps(activities)

In [ ]:
activities_prompt = get_system_prompt(
    """You are an activities recommendation agent for Japan trips. Determine season from start_date (spring: Mar-May, summer: Jun-Aug, autumn: Sep-Nov, winter: Dec-Feb).
    Focus on seasonal highlights like sakura in spring or snow in winter.
    Use arrival region from flight_info and season from start_date.
    Recommend 5-10 activities with locations, duration, best_time, description. Use tools if needed.
    Output as JSON list of dicts."""
)


In [ ]:
activities_agent = create_react_agent(
    llm,
    tools=[search_seasonal_activities],
    prompt=activities_prompt,
)

### Schedule Generate Agent

In [ ]:
@tool
def get_date_range(start_date: str, return_date: str) -> str:
    """Get list of dates in trip."""
    from datetime import datetime, timedelta
    dates = []
    current = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(return_date, '%Y-%m-%d')
    while current < end:
        dates.append(current.strftime('%Y-%m-%d'))
        current += timedelta(days=1)
    return json.dumps(dates)

In [ ]:
schedule_prompt = get_system_prompt(
    """You are a schedule agent. Using activities list, flight_info (arrival/departure times), and dates, create a daily itinerary.
    Consider location proximity, travel times (use tools), and avoid overload. 
    Output as JSON: {date: [{activity: ..., start_time: ..., location: ...}]}."""
)

In [ ]:
schedule_agent = create_react_agent(
    llm,
    tools=[get_date_range],
    prompt=schedule_prompt)

### Define Graph

In [10]:
def flight_data_node(state: State) -> State:
    start_flight = state.get('start_flight') or "extract_from_messages"
    return_flight = state.get('return_flight') or "extract_from_messages"
    
    # Run agent (assuming it calls tool and gets JSON)
    agent_output = flight_data_agent.invoke({"messages": state['messages'] + [HumanMessage(content=f"Get data for {start_flight} and {return_flight}")]})
    
    # Parse and update state
    flight_info = json.loads(agent_output['output'])  # Or similar
    state['flight_info'] = flight_info
    state['start_date'] = "parse_from_flight_info"  # e.g., extract scheduled date
    state['return_date'] = "parse_from_flight_info"
    state['messages'].append(SystemMessage(content="Flight info updated."))
    return state

In [11]:
def day_to_split_node(state: State) -> State:
    return state


In [12]:
def activities_pool_node(state: State) -> State:
    return state

In [13]:
def daily_schedule_node(state: State) -> State:
    return state

In [14]:
def total_schedule_node(state: State) -> State:
    return state

In [15]:
def report_node(state: State) -> State:
    return state

In [16]:
def join_func(state: State) -> Dict:
    if len(state['chunk_results']) == state['num_splits']:
        combined = { 
            'total_trip_schedule': sum(chunk['days'] for chunk in state['chunk_results']),  # Placeholder logic
            'all_schedules': state['chunk_results']
        }
        return {'combined_schedule': combined}  # Update state for report_agent
    else:
        return {}  # Not all done yet; noop

In [17]:
def split_router(state: State):
    return [Send("activities_pool_agent", {"current_chunk": chunk}) for chunk in state['splits']]

In [18]:
def total_schedule_func(state: State) -> Dict:
    # Compute schedule based on state['current_chunk'] (and prior steps' outputs)
    chunk_schedule = {'location': state['current_chunk']['location'], 'schedule': ...}  # Your logic
    return {'chunk_results': [chunk_schedule]}  # Note: list for reducer concatenation

In [19]:
def join_router(state: State):
    if 'combined_schedule' in state:  # Or check len(chunk_results) == num_splits again
        return "report_agent"
    else:
        return "__end__"  # End this branch early



In [20]:
workflow = StateGraph(State)
workflow.add_node("flight_data_acquisition_agent", flight_data_node)
workflow.add_node("day_to_split_agent", day_to_split_node)
workflow.add_node("join_agent", join_func)
workflow.add_node("activities_pool_agent", activities_pool_node)
workflow.add_node("daily_schedule_agent", daily_schedule_node)
workflow.add_node("total_schedule_agent", total_schedule_node)
workflow.add_node("report_agent", report_node)



# Sequential edges
workflow.add_edge("flight_data_acquisition_agent", "day_to_split_agent")
workflow.add_conditional_edges("day_to_split_agent", split_router)
workflow.add_edge("activities_pool_agent", "daily_schedule_agent")
workflow.add_edge("daily_schedule_agent", "total_schedule_agent")
workflow.add_edge("total_schedule_agent", "join_agent")
workflow.add_conditional_edges("join_agent", join_router)

workflow.add_edge("report_agent", END)

# Entry point (start with user message in state['messages'])
workflow.set_entry_point("flight_data_acquisition_agent")
graph = workflow.compile()

In [21]:
graph.get_graph().draw_png()

ImportError: Install pygraphviz to draw graphs: `pip install pygraphviz`.

### Run Graph

In [ ]:
input_message = HumanMessage(
    content="Design a travel itinerary")

In [ ]:
START_FLIGHT = "HX614"
RETURN_FLIGHT = "HX617"
START_DATE = "2024-12-20"
RETURN_DATE = "2025-01-05"

EMAIL_ADDRESS = "terence2379@gmail.com"

In [ ]:
events = graph.stream(
    input={
        "messages": [input_message],
        "start_flight": START_FLIGHT,
        "return_flight": RETURN_FLIGHT,
        "start_date": START_DATE,
        "return_date": RETURN_DATE
    },
    # Maximum number of steps to take in the graph
    config={"recursion_limit": 15},
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        message = event["messages"][-1]
        message.pretty_print()

### Send Email


In [ ]:
body = message.content

title = llm.invoke(
    f"Extract the subject from this report content as the email subject and return the title directly without any introductory text: {body}")

subject = title.content

In [ ]:
result = send_report_via_email(subject, body, EMAIL_ADDRESS)
print(result)